<a href="https://colab.research.google.com/github/pmoskovi/google-colab/blob/master/FindQueen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving im-60.jpg to im-60.jpg
Saving im-59.jpg to im-59.jpg
Saving im-58.jpg to im-58.jpg
Saving im-57.jpg to im-57.jpg
Saving im-56.jpg to im-56.jpg
Saving im-55.jpg to im-55.jpg
Saving im-11.jpg to im-11.jpg
Saving im-54.jpg to im-54.jpg
Saving im-53.jpg to im-53.jpg
Saving im-52.jpg to im-52.jpg
Saving im-51.jpg to im-51.jpg
Saving im-10.jpg to im-10.jpg
Saving im-09.jpg to im-09.jpg
Saving im-08.jpg to im-08.jpg
Saving im-07.jpg to im-07.jpg
Saving im-06.jpg to im-06.jpg
Saving im-05.jpg to im-05.jpg
Saving im-04.jpg to im-04.jpg
Saving im-03.jpg to im-03.jpg
Saving im-02.jpg to im-02.jpg
Saving im-01.jpg to im-01.jpg
User uploaded file "im-60.jpg" with length 220550 bytes
User uploaded file "im-59.jpg" with length 605369 bytes
User uploaded file "im-58.jpg" with length 45016 bytes
User uploaded file "im-57.jpg" with length 252377 bytes
User uploaded file "im-56.jpg" with length 655894 bytes
User uploaded file "im-55.jpg" with length 1089901 bytes
User uploaded file "im-11.jpg" wit

In [0]:
import tensorflow as tf


# step 1
filenames = tf.constant(['im-01.jpg', 'im-02.jpg', 'im-03.jpg', 'im-04.jpg', 'im-05.jpg', 'im-06.jpg', 'im-07.jpg', 'im-08.jpg', 'im-09.jpg', 'im-10.jpg', 'im-11.jpg', 
                         'im-51.jpg', 'im-52.jpg', 'im-53.jpg', 'im-54.jpg', 'im-55.jpg', 'im-56.jpg', 'im-57.jpg', 'im-58.jpg', 'im-59.jpg', 'im-60.jpg' ])
labels = tf.constant(
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
0,0,0,0,0,0,0,0,0,0])

# step 2: create a dataset returning slices of `filenames`
dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))

# step 3: parse every image in the dataset using `map`
def _parse_function(filename, label):
    image_string = tf.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.cast(image_decoded, tf.float32)
    return image, label

dataset = dataset.map(_parse_function)
dataset = dataset.batch(2)

# step 4: create iterator and final input tensor
iterator = dataset.make_one_shot_iterator()
images, labels = iterator.get_next()
